In [1]:
import os
import sagemaker
from sagemaker.tensorflow import TensorFlow

sagemaker_session = sagemaker.Session()

bucket = 'calvinandpogs-ee148'

role = sagemaker.get_execution_role()
print(bucket, role)
print(os.getcwd())

calvinandpogs-ee148 arn:aws:iam::652516965730:role/service-role/AmazonSageMaker-ExecutionRole-20210513T011299
/home/ec2-user/SageMaker/atrw/models/yolov5


## Yolov5s Testing

In [5]:
# TENSORFLOW CONTAINER (PY37)

image_set = 'train'

estimator = TensorFlow(entry_point='test.py',
                        source_dir='./',
                        role=role,
                        instance_count=1,
                        instance_type="ml.g4dn.xlarge",
                        framework_version="2.2",
                        py_version="py37",
                        hyperparameters={
                            'img-size': 1920,
                            'task': image_set,
                            'weights': 'SM_CHANNEL_MODEL/best.pt',
                            'data': 'atrw.yaml',
                            'anno-json': 'data/atrw_detect_center.json',
                            'output-s3': f's3://calvinandpogs-ee148/atrw/out/detection/yolov5/test-s/{image_set}',
                            'save-s3': True
                        }
)

In [ ]:
estimator.fit({'model': f's3://{bucket}/models/yolov5/train-full/',
               
               'annot': f's3://{bucket}/atrw/detection/annotations/yolov5/ImageSets/Main',
               'label': f's3://{bucket}/atrw/detection/annotations/yolov5/labels',
               'train': f's3://{bucket}/atrw/detection/train/',
               
               'annot_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/ImageSets',
               'label_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/labels',
               'test': f's3://{bucket}/atrw/detection/test/'})

## Yolov5m Testing

In [10]:
# TENSORFLOW CONTAINER (PY37)

image_set = 'train'

estimator = TensorFlow(entry_point='test.py',
                        source_dir='./',
                        role=role,
                        instance_count=1,
                        instance_type="ml.g4dn.xlarge",
                        framework_version="2.2",
                        py_version="py37",
                        hyperparameters={
                            'img-size': 1920,
                            'task': image_set,
                            'weights': 'SM_CHANNEL_MODEL/best.pt',
                            'data': 'atrw.yaml',
                            'anno-json': 'data/atrw_detect_corner.json',
                            'output-s3': f's3://calvinandpogs-ee148/atrw/out/detection/yolov5/test-m/{image_set}',
                            'save-s3': True
                        }
)

In [ ]:
estimator.fit({'model': f's3://{bucket}/models/yolov5/train-m/train-full/',
               
               'annot': f's3://{bucket}/atrw/detection/annotations/yolov5/ImageSets/Main',
               'label': f's3://{bucket}/atrw/detection/annotations/yolov5/labels',
               'train': f's3://{bucket}/atrw/detection/train/',
               
               'annot_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/ImageSets',
               'label_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/labels',
               'test': f's3://{bucket}/atrw/detection/test/'})

## Testing Subsets

### Fractional Subsets

In [9]:
%%capture --no-stderr

sets = [16, 8, 4, 2]

for set in sets:
    estimator = TensorFlow(entry_point='test.py',
                            source_dir='./',
                            role=role,
                            instance_count=1,
                            instance_type="ml.g4dn.xlarge",
                            framework_version="2.2",
                            py_version="py37",
                            hyperparameters={
                                'img-size': 1920,
                                'task': 'test',
                                'weights': 'SM_CHANNEL_MODEL/best.pt',
                                'data': f'atrw{set}.yaml',
                                'anno-json': 'data/atrw_detect_center.json',
                                'output-s3': f's3://calvinandpogs-ee148/atrw/out/detection/yolov5/test-s/frac/test{set}/',
                                'save-s3': True
                            }
    )
    estimator.fit({'model': f's3://{bucket}/models/yolov5/train-s/frac/train{set}/',
                   'annot_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/ImageSets',
                   'label_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/labels',
                   'test': f's3://{bucket}/atrw/detection/test/'})

### Clustering Split

In [7]:
%%capture --no-stderr

""" 
Image Sets:
    train: test on the training split
    val: test on the validation split
    test-split: test on the test split
    test: test on the full test set (from ATRW)
"""
image_set = 'train'
image_task = 'test' if image_set == 'test-split' else image_set

sets = list(range(10))        # 1 set per cluster split
for set in sets:
    estimator = TensorFlow(entry_point='test.py',
                            source_dir='./',
                            role=role,
                            instance_count=1,
                            instance_type="ml.g4dn.xlarge",
                            framework_version="2.2",
                            py_version="py37",
                            hyperparameters={
                                'img-size': 1920,
                                'task': image_task,
                                'weights': 'SM_CHANNEL_MODEL/best.pt',
                                'data': 'atrw.yaml',
                                'anno-json': 'data/atrw_detect_center.json',
                                'output-s3': f's3://calvinandpogs-ee148/atrw/out/detection/yolov5/test-s/clusters/split{set}/{image_set}',
                                'save-s3': True
                            }
    )
    
    if image_set =='test-split':
        estimator.fit({'model': f's3://{bucket}/models/yolov5/train-s/clusters/split{set}/',

                       'annot': f's3://{bucket}/atrw/detection/annotations/yolov5/ImageSets/clustering/split{set}',
                       'label': f's3://{bucket}/atrw/detection/annotations/yolov5/labels',
                       'train': f's3://{bucket}/atrw/detection/train/',

                       'annot_test': f's3://{bucket}/atrw/detection/annotations/yolov5/ImageSets/clustering/split{set}',
                       'label_test': f's3://{bucket}/atrw/detection/annotations/yolov5/labels',
                       'test': f's3://{bucket}/atrw/detection/train/'})
    else:
        estimator.fit({'model': f's3://{bucket}/models/yolov5/train-s/clusters/split{set}/',

                       'annot': f's3://{bucket}/atrw/detection/annotations/yolov5/ImageSets/clustering/split{set}',
                       'label': f's3://{bucket}/atrw/detection/annotations/yolov5/labels',
                       'train': f's3://{bucket}/atrw/detection/train/',

                       'annot_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/ImageSets',
                       'label_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/labels',
                       'test': f's3://{bucket}/atrw/detection/test/'})